In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        dir = os.path.join(dirname, filename)
        all_data = pd.read_csv(dir)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Colocamos las semillas para poder reproducir los experimentos
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
#Con head vemos los 5 primeros registros
all_data.head()

In [ ]:
all_data.describe()

In [ ]:
#Podemos ver algunos datos interesantes con describe()
df_maximos = all_data.describe().iloc[-1]
maximos = []

for i in range(0, len(df_maximos)):
    maximos.append(df_maximos[i])

In [ ]:
columnas = all_data.columns
print(columnas)

In [ ]:
for i, c in enumerate(columnas[1:]):
    plt.hist(all_data[c])
    plt.xticks(rotation = 'vertical')
    
    # Así puedes especificar el número de xticks
    plt.xticks(np.linspace(0,maximos[i],26))
    
    plt.title(c)
    plt.show()

In [ ]:
print(all_data.corr())
sns.heatmap(all_data.corr())

In [ ]:
#Como podemos ver no hay nulos en este dataset
all_data.isna().sum()

In [ ]:
df = all_data.filter(['Close'])
df

In [ ]:
import math

initial_moment = datetime.datetime(2015, 11, 23)

training_len = math.ceil(0.8 * len(df))
training = df.copy()
training = training.iloc[:training_len]

test = df.copy()
test = test.iloc[training_len:]

In [ ]:
print("Data Shapes")
print("Shape Training:", training.shape)
print("Shape Test:", test.shape)

all_data.iloc[1461]

In [ ]:
# La evolución del precio por días
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,6))

x = [initial_moment + datetime.timedelta(days=j) for j in range(all_data.shape[0])]
for row in columnas[1:]:
    if row != 'Volume':
        ax.plot(x, all_data[row], label = row)

ax.set_title('Close vs High vs Low vs Open')
plt.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,6))

x = [initial_moment + datetime.timedelta(days=j) for j in range(all_data.shape[0])]
ax.plot(x, all_data['Volume'], label = 'Volume')

ax.set_title('Volume data')
plt.legend()
plt.show()

In [ ]:
# La evolución del precio por días
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,6))

x = [initial_moment + datetime.timedelta(days=j) for j in range(training.shape[0])]
ax.plot(x, training['Close'], label = 'Close')

ax.set_title('Price Yahoo')
plt.legend()
plt.show()

In [ ]:
# La evolución del precio por días
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,6))
initial_moment_test = datetime.datetime(2019, 1, 23)
x = [initial_moment_test + datetime.timedelta(days=j) for j  in range(test.shape[0])]
ax.plot(x, test['Close'])

ax.set_title('Price Yahoo Test Data')
plt.show()

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df)

scaled_data

In [ ]:
training = scaled_data[0:training_len]
test = scaled_data[training_len-60:]

In [ ]:
x_train = []
y_train = []

for i in range(60, len(training)):
    x_train.append(training[i-60:i])
    y_train.append(training[i])
    
x_train = np.asarray(x_train)
y_train = np.asarray(y_train)

In [ ]:
x_test = []
y_test = []

for i in range(60, len(test)):
    x_test.append(test[i-60:i])
    y_test.append(test[i])
    
x_test = np.asarray(x_test)
y_test = np.asarray(y_test)

In [ ]:
x_train.shape

In [ ]:
# LSTM
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, LSTM, Input
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPool1D, LSTM, Input, GRU
from tensorflow.keras import backend as K
from collections import defaultdict
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.metrics import mean_absolute_error
import datetime
import random
import math

In [ ]:
# Create model with one LSTM layer with 64 units
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

In [ ]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()

In [ ]:
print("DATA SHAPES")
print("x_train:", x_train.shape)
print("y_train:", y_train.shape)
print()
print("x_test:", x_test.shape)
print("y_test:", y_test.shape)

In [ ]:
history = model.fit(x_train, y_train,
          batch_size=256,
          epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
predictions = model.predict(x_test)
predictions_scaled = scaler.inverse_transform(predictions)

In [ ]:
predictions_scaled

In [ ]:
rmse = np.sqrt(np.mean(predictions - y_test)**2)
rmse

In [ ]:
train = all_data[:training_len]
valid = all_data[training_len:]
valid['Predictions'] = predictions_scaled

In [ ]:
valid

In [ ]:
import datetime
initial_moment = datetime.datetime(2015, 11, 23)

In [ ]:
x = [initial_moment + datetime.timedelta(days=j) for j in range(training.shape[0])]
y = [x[-1] + datetime.timedelta(days=j) for j in range(valid.shape[0])]

plt.figure(figsize=(16,8))
plt.title("Model")
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)

plt.plot(x, train['Close'])
plt.plot(y, valid[['Close', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'])
plt.show()

In [ ]:
x_train.shape

In [ ]:
predictions_scaledl = []
for i in range(0,5):
    x_train = x_train[0:359]
    predictions = np.reshape(predictions[5:], (6 ,60,1))
    x_train = np.concatenate((x_train, predictions))
    
    predictions = model.predict(x_train)
    predictions_scaled = scaler.inverse_transform(predictions)
    predictions_scaledl = np.append(predictions_scaledl, predictions_scaled)

long_predic = pd.DataFrame({'newPred': predictions_scaledl})

In [ ]:
initial_moment = datetime.datetime(2015, 11, 23)

x = [initial_moment + datetime.timedelta(days=j) for j in range(training.shape[0])]
y = [x[-1] + datetime.timedelta(days=j) for j in range(len(valid.Predictions))]
z = [y[-1] + datetime.timedelta(days=j) for j in range(len(long_predic.newPred))]

plt.figure(figsize=(16,8))
plt.title("Model")
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)

plt.plot(x, train['Close'])
plt.plot(y, valid[['Close', 'Predictions']])
plt.plot(z, long_predic['newPred'])
plt.legend(['Train', 'Val', 'Predictions', 'nP'])
plt.show()